# 分布鲁棒优化

**作者：**  
Daniel Kuhn  
风险分析与优化主席，  
洛桑联邦理工大学，瑞士洛桑  
电子邮件：[daniel.kuhn@epfl.ch](mailto:daniel.kuhn@epfl.ch)

Soroosh Shafiee  
康奈尔大学，运营研究与信息工程学院，  
美国纽约伊萨卡  
电子邮件：[shafiee@cornell.edu](mailto:shafiee@cornell.edu)

Wolfram Wiesemann  
帝国理工学院商学院，  
英国伦敦帝国理工大学  
电子邮件：[ww@imperial.ac.uk](mailto:ww@imperial.ac.uk)

---

## 摘要

分布鲁棒优化（DRO）研究不确定性下的决策问题，其中支配不确定问题参数的概率分布本身也是不确定的。任何DRO模型的一个关键组成部分是其模糊集，即与任何可用的结构或统计信息一致的概率分布族。DRO寻找在模糊集中的最坏分布下表现最好的决策。这一最坏情况标准得到了心理学和神经科学研究的支持，这些研究表明，许多决策者对分布不确定性有较低的容忍度。DRO根植于统计学、运筹学和控制理论，近期研究揭示了它与机器学习中的正则化技术和对抗训练之间的深刻联系。本综述以统一且自成体系的方式呈现了该领域的关键发现。

---

## 目录

1. [引言](#引言)
2. [模糊集](#模糊集)
3. [模糊集的拓扑属性](#模糊集的拓扑属性)
4. [最坏期望问题的对偶理论](#最坏期望问题的对偶理论)
5. [最坏风险问题的对偶理论](#最坏风险问题的对偶理论)
6. [自然子问题的解析解](#自然子问题的解析解)
7. [自然子问题的有限凸重构](#自然子问题的有限凸重构)
8. [通过鲁棒化进行正则化](#通过鲁棒化进行正则化)
9. [DRO问题的数值求解方法](#dro问题的数值求解方法)
10. [统计保证](#统计保证)
11. [参考文献](#参考文献)


## 1. 引言 (Introduction)
传统上，数学优化研究的是以下形式的问题：

$$
\inf _{x \in \mathcal{X}} \ell(x),
$$

其中，决策变量 $x$ 从可行解集 $\mathcal{X} \subseteq \mathbb{R}^n$ 中选择，目的是最小化损失函数 $\ell: \mathbb{R}^n \rightarrow \overline{\mathbb{R}}$。数学优化起源于17世纪末艾萨克·牛顿、戈特弗里德·威廉·莱布尼茨、皮埃尔·德·费马等人对微积分的研究，至今已有丰富的历史，涉及许多数学家、经济学家、工程师和科学家的贡献。现代数学优化的诞生通常归功于乔治·丹茨格，他在1947年提出的单纯形法可以求解线性优化问题，其中损失函数 $\ell$ 是仿射的，且可行集 $\mathcal{X}$ 是一个多面体（Dantzig 1956）。随后的重要进展包括：凸分析的丰富理论（Rockafellar 1970），以及线性优化问题的多项式时间求解方法（Khachiyan 1979，Karmarkar 1984）和广泛类别的非线性凸优化问题（Nesterov 和 Nemirovskii 1994）。

经典的优化问题是确定性的，即假设所有问题数据都是已知且确定的。然而，实践中遇到的大多数决策问题都依赖于那些受到测量误差影响的参数，或者这些参数仅在决策必须做出并且被确认之后才能揭示。将不确定性影响的决策问题简单地建模为确定性优化问题的一种方法是用它们的期望值或适当的点预测值替代所有不确定的参数。然而，早在很久以前就有人发现并记录了这样一个事实：那些将不确定的参数替换为其均值的决策者往往会陷入“平均误差”的陷阱（Savage, Scholtes 和 Zweidler 2006，Savage 2012）。为了考虑那些偏离均值的不确定性实现，Beale（1955）和Dantzig（1955）独立提出了以下形式的随机规划：

$$
\inf _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)],
$$

这些方法明确地将不确定的问题参数 $Z$ 建模为一个由概率分布 $\mathbb{P}$ 支配的随机向量，并且在此基础上寻找在期望下表现最好的决策（或根据某些风险度量进行优化）。此后，随机规划已发展成为一个成熟的领域（Birge 和 Louveaux 2011，Shapiro, Dentcheva 和 Ruszczyński 2009），并为机器学习中的经验风险最小化原则提供了理论基础（Bishop 2006，Hastie, Tibshirani 和 Friedman 2009）。

尽管随机规划在理论和实践中取得了成功，但它至少存在两个缺点。首先，假设概率分布 $\mathbb{P}$ 是完全已知的，在许多实际情况下是不现实的，且随机规划对这种分布的误设非常敏感。不同领域对这一效应的描述有：优化者的诅咒（Smith 和 Winkler 2006），优化的误差最大化效应（Michaud 1989，DeMiguel 和 Nogales 2009），优化偏差（Shapiro 2003）或过拟合（Bishop 2006，Hastie 等 2009）。其次，评估固定决策的期望损失需要计算多维积分，而这对于非常简单的损失函数和分布来说也已经是一个非常复杂的问题。因此，随机规划受到维度灾难的困扰，即它们的计算复杂度通常在随机向量 $Z$ 的维度上呈指数增长。为了缓解这两个缺点，Soyster（1973）提出将不确定性影响的决策问题建模为鲁棒优化问题，形式为：

$$
\inf _{x \in \mathcal{X}} \sup _{z \in \mathcal{Z}} \ell(x, z).
$$

鲁棒优化用基于集合的描述代替了不确定问题参数的概率描述，寻找在最坏情况下的参数实现 $z$ 下表现最好的决策。经过长时间的忽视，Soyster（1973）的思想在90年代末得到了重新审视并大大扩展，Kouvelis 和 Yu（1997），El Ghaoui、Oustry 和 Lebret（1998），El Ghaoui 和 Lebret（1998a,b），BenTal 和 Nemirovski（1999b, 1998, 1999a），Bertsimas 和 Sim（2004）等人进一步推进了该领域。有关鲁棒优化文献的综述，请参见 Ben-Tal、El Ghaoui 和 Nemirovski（2009），Rustem 和 Howe（2009）和 Bertsimas 和 den Hertog（2022）。我们指出，类似的思想在鲁棒稳定性（Horn 和 Johnson 1985，Doyle、Glover、Khargonekar 和 Francis 1989，Green 和 Limebeer 1995）和鲁棒控制（Zames 1966，Khalil 1996，Zhou、Doyle 和 Glover 1996）领域中也得到了独立发展，这些领域研究系统在参数变化下是否能够保持稳定，或者在参数变化下设计出具有良好性能的系统。有关鲁棒稳定性和控制的教材介绍，请参见 Zhou 和 Doyle（1999）和 Dullerud 和 Paganini（2001）。Hansen 和 Sargent（2008）将鲁棒控制技术应用于受模型不确定性影响的经济问题，设计在多种可能的模型误设下都能表现良好的政策。

尽管鲁棒优化减少了随机规划中的信息和计算负担，但它对模糊集内所有参数实现的平等对待及其对最坏情境的专注，可能使得它在实际应用中过于保守。这些问题促使研究人员研究分布鲁棒优化问题，形式为：

$$
\inf _{x \in \mathcal{X}} \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)],
$$

该模型将不确定的参数$Z$建模为一个随机向量，其由一个模糊集$\mathcal{P}$中的某个分布$\mathbb{P}$支配，寻求在最坏分布$\mathbb{P} \in \mathcal{P}$下，其期望值表现最好的决策。分布鲁棒优化（DRO）因此将随机编程的分布视角与鲁棒优化的最坏情况聚焦结合在一起。Herbert E. Scarf通常被认为是这一方法的先驱，他在关于新闻供应商问题的研究中提出，未知的需求分布仅通过其均值和方差来表征（Scarf 1958）。随后，Dupačová（1966，1987，1994）和Shapiro与Kleywegt（2002）研究了那些模糊集规定了未知概率分布的支持集、某些低阶矩、独立模式或其他结构特性的问题。Ermoliev、Gaivoronski 和 Nedeva（1985）及 Gaivoronski（1991）为基于矩模糊集的DRO问题开发了早期的求解方法。现代DRO的出现通常归功于Bertsimas和Popescu（2002，2005）的工作，他们在部分分布信息下推导了概率不等式，并将其技术应用于期权定价问题，El Ghaoui、Oks和Oustry（2003）以及Calafiore和El Ghaoui（2006）的研究，后者研究了DRO问题，其中目标函数的某个分位数应当最小化，或一组受不确定性影响的约束应当以高概率满足，跨所有具有已知矩约束的概率分布，以及Delage和Ye（2010）的研究，后者研究了类似的DRO问题，目标为最坏情况下的期望值目标。

早期关于DRO的研究主要集中在矩模糊集上，这些模糊集包含所有满足有限多个矩约束的指定支持集$\mathcal{Z}$上的分布。与随机规划不同，具有矩模糊集的DRO问题有时会在随机向量$Z$的维度上展现出有利的扩展性。然而，截然不同的分布可能具有相同的矩。因此，矩模糊集总是包含广泛的分布，其中包括一些不太可能的分布，当有充足的历史数据时，这些不太可能的分布可以安全地被排除。这促使Ben-Tal、den Hertog、De Waegenaere、Melenberg和Rennen（2013）以及Wang、Glynn和Ye（2016）引入了模糊集，这些模糊集包含所有在某个参考分布的邻域内的分布（通常是从历史数据形成的经验分布）。这些邻域可以通过概率分布之间的差异函数来定义，例如$\phi$-散度（Csiszar 1963）或Wasserstein距离（Villani 2008）。与矩模糊集不同，基于差异的模糊集具有可调的大小参数（例如，半径），因此可以缩小为仅包含参考分布的单例。如果参考分布收敛到未知的真实分布，并且随着更多历史数据的增加，大小参数趋近于0，则DRO问题最终会简化为在真实分布下的经典随机规划。早期关于基于差异的模糊集的研究假定$Z$是一个离散随机向量，具有有限的支持集$\mathcal{Z}$。基于差异的DRO问题扩展到一般的（可能是连续）随机向量的工作由Mohajerin Esfahani 和 Kuhn（2018），Zhao 和 Guan（2018），Blanchet 和 Murthy（2019），Zhang、Yang 和 Gao（2024b）以及 Gao 和 Kleywegt（2023）完成，他们使用最优传输差异构建了模糊集。我们请参阅Kuhn、Mohajerin Esfahani、Nguyen 和 Shafieezadeh-Abadeh（2019）以及Rahimian 和 Mehrotra（2022）以获取DRO文献的先前综述。

“分布鲁棒性”一词最早源于鲁棒统计。Huber（1981）首次提出该术语，用以描述在数据中存在异常值时做出鲁棒决策的方法。这个概念扩展了Box（1953，1979）的早期工作，后者研究了在基础分布偏离正态分布时的鲁棒性，正态分布是许多统计模型中的常见假设。为了应对异常值带来的挑战，统计学家们发展了几种污染模型，每个模型提供了不同的解决数据异常的途径。Huber污染模型由Huber（1964，1968）提出，并由Hampel（1968，1971）进一步发展，假设观察到的数据是从真实分布和任意污染分布的混合中抽取的。邻域污染模型通过统计距离定义偏离真实分布的情况，例如总变差（Donoho 和 Liu 1988）或Wasserstein距离（Zhu、Jiao 和 Steinhardt 2022a，Liu 和 Loh 2023）。最近，数据依赖的自适应污染模型允许将部分观察到的数据点替换为从任意分布中抽取的点（Diakonikolas、Kamath、Kane、Li、Moitra 和 Stewart 2019，Zhu 等 2022a）。有趣的是，DRO模型的乐观对偶模型，优化的是模糊集中的最佳分布（而非最坏分布），恢复了许多来自鲁棒统计的估计量（Blanchet、Li、Lin 和 Zhang 2024b，Jiang 和 Xie 2024）。关于算法鲁棒统计的最新进展，请参阅Diakonikolas 和 Kane（2023）。

鲁棒优化和分布鲁棒优化在机器学习中找到了广泛的应用。例如，机器学习文献中的流行正则化器已知具有鲁棒性解释，这为正则化在实践中的强大经验表现提供了理论洞察（Xu，Caramanis 和 Mannor 2009，Shafieezadeh-Abadeh、Kuhn 和 Mohajerin Esfahani 2019，Li、Lin、Blanchet 和 Nguyen 2022，Gao、Chen 和 Kleywegt 2024）。同样，DRO模型的乐观对偶模型，优化的是模糊集中的最佳（而非最坏）分布，产生了上置信区间算法，这些算法在赌博和强化学习文献中非常普遍（Blanchet 等 2024b，Jiang 和 Xie 2024）。DRO也与对抗训练有关，对抗训练旨在通过在对抗性样本的视角下训练模型，从而提高机器学习模型的泛化性能（Goodfellow、Shlens 和 Szegedy 2015）。对抗样本是现有数据点的扰动，旨在误导模型做出不正确的预测。


DRO 与扩展的随机（动态）编程之间也存在深刻的联系，这些扩展通过用一致的风险度量代替期望值。与期望值相似，风险度量将随机变量映射到扩展实数上。与期望值不同，期望值是风险中立的，因为它对正负结果赋予相同的权重，而风险度量通常赋予负面结果更大的权重，从而考虑到决策者经常表现出的风险厌恶。Artzner、Delbaen、Eber 和 Heath (1999) 以及 Delbaen (2002) 表明，满足一致性公理以及 Fatou 性质的风险度量可以等效地表示为在特定分布集上的最坏情况期望值。换句话说，优化最坏情况期望值（如DRO中所做的）和优化一致风险度量之间有直接的联系。在金融数学文献中，针对一类非线性期望——所谓的 $G$-期望，已经发展了类似的表示定理，这些期望基于反向随机微分方程的解（Peng 1997, 2007a,b, 2019）。Peng (2023) 显示，次线性 $G$-期望等价于在特定分布族上的最坏情况期望值，从而在 $G$-期望理论和DRO之间架起了桥梁。

从哲学上讲，DRO 与模糊厌恶原则相关，在这种原则下，个体更喜欢已知风险而不是未知风险，即使未知风险可能带来更高的回报。在经济学文献中，风险性结果的概率已知与模糊性结果的概率（部分）未知之间的区别可以追溯到至少凯恩斯（1921）和奈特（1921）。模糊厌恶的概念通过Ellsberg悖论（Ellsberg 1961）广泛传播，这是一种思想实验，在该实验中，人们被要求在一个已知分布的彩色球（例如50个红球和50个蓝球）的 urn 和一个具有未知分布的相同彩色球的 urn 之间做出选择（即红蓝球的比例未知）。尽管可能有相等或更好的赔率，许多人仍然更愿意在已知分布的 urn 上下注，即他们表现出模糊厌恶。Ellsberg悖论挑战了经典的期望效用理论，它导致了如最大最小期望效用理论（Gilboa 和 Schmeidler 1989）等扩展，作为DRO的理论基础。模糊厌恶随后在金融市场（Epstein 和 Miao 2003，Bossaerts、Ghirardato、Guarnaschelli 和 Zame 2010）、保险市场（Cabantous 2007）、个人决策（Dimmock、Kouwenberg 和 Wakker 2016）、宏观经济政策（Hansen 和 Sargent 2010）、拍卖（Salo 和 Weber 1995）以及信任博弈（Li、Turmunkh 和 Wakker 2019b）中被发现。

同时，也有大量医学和神经科学证据支持模糊厌恶的存在。Hsu、Bhatt、Adolphs、Tranel 和 Camerer（2005）发现，当个体面临模糊性时，大脑中的关键情绪处理中心——杏仁体的活动比面对已知概率的情境时更为活跃，这表明杏仁体在驱动模糊厌恶中的作用。Krain、Wilson、Arbuckle、Castellanos 和 Milham（2006）的荟萃分析突出了前额叶皮层的作用，该区域负责高级认知控制、理性决策和情绪调节，用于处理模糊性。此外，Wu、Sun、Camilleri、Eickhoff 和 Yu（2021）进行的荟萃分析表明，风险和模糊的处理都依赖于前岛叶。风险处理还会激活背内侧前额叶皮层和腹侧纹状体，而模糊处理则特定地涉及背外侧前额叶皮层、下顶叶和右前岛叶。这支持了这样一个观点：当个体面对模糊决策与风险决策时，涉及的是不同的神经机制。遗传因素可能影响个体对模糊厌恶的倾向。He、Xue、Chen、Lu、Dong、Lei、Ding、Li、Li、Chen、Li、Moyzis 和 Bechara（2010）将某些遗传多态性与个体在风险和模糊决策下的表现联系起来。在另一项研究中，Buckert、Schwieren、Kudielka 和 Fiebach（2014）研究了荷尔蒙变化（例如，与压力和焦虑相关的皮质醇水平升高）如何影响风险和模糊决策。这些发现共同表明，风险和模糊的感知不仅仅是认知现象，还受到大脑结构、遗传和荷尔蒙因素的影响，这些因素塑造了个体在模糊性决策中的差异。最后，我们提到Hartley 和 Somerville（2015）以及 Blankenstein、Crone、van den Bos 和 van Duijvenvoorde（2016），他们研究了模糊厌恶如何在儿童、青少年和成人之间有所不同，Hayden、Heilbronner 和 Platt（2010）观察到，当恒河猕猴在选择风险和模糊游戏时，表现出模糊厌恶。

本综述的其余部分结构如下。我们的分析的一个重要部分是研究最坏期望值 $\sup _{\mathrm{P} \in \mathcal{P}} \mathbb{E}_{\mathrm{P}}[\ell(x, Z)]$，这是DRO问题（1.2）的目标函数。评估这个表达式通常需要求解一个半无穷优化问题，这个问题有无限多个变量来表征概率分布 $P$，并且受到模糊集 $\mathcal{P}$ 施加的有限个约束。我们称这个问题为“自然的子问题”，它是区分DRO问题（1.2）与确定性、随机和鲁棒优化问题的关键特征。第2和第3节回顾了常见的模糊集 $\mathcal{P}$ 及其拓扑性质，特别关注自然子问题在什么条件下能够达到最优值。第4和第5节为自然子问题开发了对偶性理论，使我们能够上界或等价地重新表述最坏期望值，使用一个半无穷优化问题，通过有限多个对偶决策变量，施加无限多个约束。这个对偶性框架为第6节自然子问题的解析解奠定了基础，后者依赖于构建原始和对偶可行解，二者产生相同的目标值，从而享有强对偶性。第7和第8节利用相同的对偶性理论，开发了自然子问题以及整个DRO问题（1.2）的等价重构和保守逼近。第9节展示了对偶性理论如何为自然子问题和完整DRO问题提供数值求解技术。最后，第10节回顾了不同模糊集所享有的统计保障。

由于篇幅限制，本综述在选择涵盖的主题时面临艰难的取舍。我们决定聚焦于最常用的模糊集，并简要回顾其他可能的选择，例如边际模糊集、具有结构约束的模糊集（包括对称性和单峰性）、Sinkhorn模糊集或条件相对熵模糊集。同样，我们没有涵盖分布优先优化和决策随机化等重要但略显复杂的话题。最后，我们专注于单阶段问题，其中不确定性在作出此时此刻的决策 $x \in \mathcal{X}$ 后完全解决；两阶段和多阶段DRO问题，其中不确定性随时间展开并且可以进行补救决策，已由 Delage 和 Iancu（2015）以及 Yanikoglu、Gorissen 和 den Hertog（2019）进行了综述。





## 1.1 符号

本文中考虑的所有向量空间都定义在实数域上。为了简洁起见，我们简单地称它们为“向量空间”而不是“实向量空间”。我们使用 $\overline{\mathbb{R}} = \mathbb{R} \cup \{-\infty, \infty\}$ 来表示扩展实数。函数 $f: \mathbb{R}^d \rightarrow \overline{\mathbb{R}}$ 的有效域定义为 $\operatorname{dom}(f) = \{z \in \mathbb{R}^d : f(z) < \infty\}$，而 $f$ 的上图定义为 $\operatorname{epi}(f) = \{(z, \alpha) \in \mathbb{R}^d \times \mathbb{R}: f(z) \leq \alpha\}$。我们称 $f$ 为适当的，当且仅当 $\operatorname{dom}(f) \neq \emptyset$ 且对于所有 $z \in \mathbb{R}^d$，$f(z) > -\infty$。$f$ 的凸共轭是函数 $f^*: \mathbb{R}^d \rightarrow \overline{\mathbb{R}}$，通过 $f^*(y) = \sup_{z \in \mathbb{R}^d} y^{\top} z - f(z)$ 定义。一个凸函数 $f$ 被称为闭合的，如果它是适当的并且下半连续，或者它恒等于 $+\infty$ 或 $-\infty$。可以证明，$f$ 是闭合的当且仅当它与其双共轭 $f^{**}$ 一致，即与 $f^*$ 的共轭一致。如果 $f$ 是适当的、凸的且下半连续的，那么它的退化函数 $f^{\infty}: \mathbb{R}^d \rightarrow \mathbb{R}$ 通过以下公式定义：$f^{\infty}(z) = \lim_{\alpha \to \infty} \alpha^{-1}(f(z_0 + \alpha z) - f(z_0))$，其中 $z_0$ 是 $\operatorname{dom}(f)$ 中的任意一点（Rockafellar 1970，第8.5定理）。$f$ 的透视图是函数 $f^\pi: \mathbb{R}^d \times \mathbb{R} \rightarrow \overline{\mathbb{R}}$，通过以下公式定义：$f^\pi(z, t) = t f(z / t)$（当 $t > 0$ 时），$f^\pi(z, t) = f^{\infty}(z)$（当 $t = 0$ 时），$f^\pi(z, t) = \infty$（当 $t < 0$ 时）。可以证明，$f^\pi$ 是适当的、凸的且下半连续的（Rockafellar 1970，第67页）。在没有混淆的风险时，我们偶尔使用 $t f(z / t)$ 来表示 $f^\pi(z, t)$，即使 $t = 0$。集合 $\mathcal{Z} \subseteq \mathbb{R}^d$ 的指示函数 $\delta_{\mathcal{Z}}: \mathbb{R}^d \rightarrow \mathbb{R}$ 定义为：如果 $z \in \mathcal{Z}$，则 $\delta_{\mathcal{Z}}(z) = 0$；如果 $z \notin \mathcal{Z}$，则 $\delta_{\mathcal{Z}}(z) = \infty$。$\delta_{\mathcal{Z}}$ 的共轭 $\delta_{\mathcal{Z}}^*$ 被称为 $\mathcal{Z}$ 的支持函数。它满足 $\delta_{\mathcal{Z}}^*(y) = \sup_{z \in \mathcal{Z}} y^{\top} z$。随机对象用大写字母表示（例如，$Z$），其实现用小写字母表示（例如，$z$）。对于任意闭集 $\mathcal{Z} \subseteq \mathbb{R}^d$，我们使用 $\mathcal{M}(\mathcal{Z})$ 来表示所有有限签名的Borel测度空间，而 $\mathcal{M}_+(\mathcal{Z})$ 表示 $\mathcal{M}(\mathcal{Z})$ 中所有（非负的）Borel测度的凸锥，$\mathcal{P}(\mathcal{Z})$ 表示 $\mathcal{M}_+(\mathcal{Z})$ 中所有概率分布的凸集。关于 $P \in \mathcal{P}(\mathcal{Z})$ 的期望算子定义为：$\mathbb{E}_{P}[f(Z)] = \int_{\mathcal{Z}} f(z) \mathbb{dP}(z)$，对于任何 Borel 函数 $f: \mathcal{Z} \rightarrow \overline{\mathbb{R}}$。如果 $f$ 的正负部分的积分都为 $\infty$，则我们“敌对”地定义 $\mathbb{E}_{P}[f(Z)]$。即，如果积分出现在最小化（最大化）问题的目标函数中，则我们设定 $\mathbb{E}_{P}[f(Z)] = \infty$（或 $-\infty$）。Dirac 概率分布将单位概率赋给 $z \in \mathcal{Z}$，记作 $\delta_z$。Dirac 分布 $\delta_z$ 不应与单点集合 $\{z\}$ 的指示函数 $\delta_{\{z\}}$ 混淆。对于任何 $\mathcal{P} \in \mathcal{P}(\mathcal{Z})$ 和任何 Borel 可测变换 $f: \mathcal{Z} \rightarrow \mathcal{Z}^{\prime}$，其中 $\mathcal{Z} \subseteq \mathbb{R}^d$ 和 $\mathcal{Z}^{\prime} \subseteq \mathbb{R}^{d^{\prime}}$，我们用 $\mathbb{P} \circ f^{-1}$ 表示在 $f$ 下的推前分布。因此，如果 $Z$ 是一个由 $P$ 支配的在 $\mathcal{Z}$ 上的随机向量，那么 $f(Z)$ 就是一个由 $\mathbb{P} \circ f^{-1}$ 支配的在 $\mathcal{Z}^{\prime}$ 上的随机向量。一个集合 $\mathcal{Z} \subseteq \mathbb{R}^d$ 的闭包、内点和相对内点分别用 $\mathrm{cl}(\mathcal{Z})$、$\operatorname{int}(\mathcal{Z})$ 和 $\operatorname{rint}(\mathcal{Z})$ 表示。我们使用 $\mathbb{R}_+^d$ 和 $\mathbb{R}_{++}^d$ 来表示 $\mathbb{R}^d$ 中的非负正交体及其内点。此外，我们使用 $\mathbb{S}^d$ 来表示所有对称矩阵的空间，$\mathbb{R}^{d \times d}$ 中的对称矩阵。正半定矩阵的锥体用 $\mathbb{S}_+^d$ 表示，而 $\mathbb{S}_{++}^d$ 表示其内点，即所有正定矩阵的集合。逻辑命题 $\mathbb{1}_{\mathcal{E}}$ 的真值为1，当且仅当 $\mathcal{E}$ 为真，否则为0。所有自然数集合 $\{1, 2, 3, \ldots\}$ 用 $\mathbb{N}$ 表示，$[n] = \{1, \ldots, n\}$ 表示所有小于等于 $n$ 的整数的集合。

## 2. 模糊集合

模糊集合 $\mathcal{P}$ 是一个概率分布族，定义在一个公共的可测空间上。本文中，我们假设 $\mathcal{P} \subseteq \mathcal{P}(\mathcal{Z})$，其中 $\mathcal{P}(\mathcal{Z})$ 表示所有 Borel 概率分布的全集，这些分布定义在闭集 $\mathcal{Z} \subseteq \mathbb{R}^d$ 上。本节回顾了几种常见的模糊集合类别。对于每个类别，我们首先给出正式定义并提供历史背景信息。随后，我们举出重要的模糊集合实例，并突出它们的应用。




### 2.1. 矩量模糊集合

矩量模糊集合是一个满足有限个（广义的）矩量条件的概率分布族。形式上，它可以表示为：

$$
\mathcal{P}=\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{E}_{\mathrm{P}}[f(\mathrm{Z})] \in \mathcal{F}\right\},
$$

其中 $f: \mathcal{Z} \rightarrow \mathbb{R}^m$ 是一个 Borel 可测矩量函数，$\mathcal{F} \subseteq \mathbb{R}^m$ 是一个不确定性集合。根据定义，矩量模糊集合（2.1）包含所有支持在 $\mathcal{Z}$ 上的概率分布 $P$，其广义矩量 $\mathbb{E}_{\mathrm{P}}[f(Z)]$ 是定义良好的，并且属于不确定性集合 $\mathcal{F}$。类型为（2.1）的模糊集合首次由 Isii $(1960, 1962)$ 和 Karlin 与 Studden (1966) 研究，用于建立广义切比雪夫不等式的精确度。以下小节回顾了矩量模糊集合的常见实例。


#### 2.1.1. 仅支持模糊集合

仅支持模糊集合包含所有支持在 $\mathcal{Z} \subseteq \mathbb{R}^d$ 上的概率分布，即 $\mathcal{P} = \mathcal{P}(\mathcal{Z})$。它可以视为（2.1）的一种实例，设 $f(z) = 1$ 且 $\mathcal{F} = \{1\}$。任何具有模糊集合 $\mathcal{P}(\mathcal{Z})$ 的 DRO 问题，本质上都等同于一个经典的鲁棒优化问题，其不确定性集合为 $\mathcal{Z}$，即：

$$
\inf _{x \in \mathcal{X}} \sup _{P \in \mathcal{P}(\mathcal{Z})} \mathrm{B}_{\mathrm{P}}[\ell(x, Z)] = \inf _{x \in \mathcal{X}} \sup _{z \in \mathcal{Z}} \ell(x, z) .
$$

有关鲁棒优化的理论和应用的综合回顾，请参见 (Ben-Tal 和 Nemirovski 1998, 1999a, 2000, 2002, Bertsimas 和 Sim 2004, Ben-Tal 等人 2009, Bertsimas, Brown 和 Caramanis 2011, Ben-Tal, den Hertog 和 Vial 2015a, Bertsimas 和 den Hertog 2022)。

如果不确定集合 $\mathcal{Z}$ 覆盖了某个分布 $\mathbb{P}$ 的总概率质量的 $1-\varepsilon$ 的一部分，那么最坏情况损失 $\sup _{z \in \mathcal{Z}} \ell(x, z)$ 保证超过在 $\mathbb{P}$ 下 $\ell(x, Z)$ 的 $(1-\varepsilon)$-分位数。这可以通过利用 $\mathbb{P}$ 的先验结构信息或统计数据来实现。例如，如果 $\mathbb{P}(Z \in \mathcal{Z}) \geq 1-\varepsilon$（确定成立），则可以通过构造一个合适大小的半空间和椭球体的交集来满足这一条件，且如果 $Z$ 在 $\mathbb{P}$ 下具有独立的、对称的、单峰的和/或次高斯分量（Bertsimas and Sim 2004, Janak, Lin and Floudas 2007, BenTal et al. 2009, Li, Ding and Floudas 2011, Bertsimas, den Hertog and Pauphilet 2021）。或者，如果 $\mathcal{Z}$ 是通过使用统计假设检验（Postek, den Hertog and Melenberg 2016, Bertsimas, Gupta and Kallus 2018b,a）、分位数估计（Hong, Huang and Lam 2021）或基于学习的方法（Han, Shang and Huang 2021, Goerigk and Kurtz 2023, Wang, Becker, Van Parys and Stellato 2023）从 $\mathbb{P}$ 的独立样本中构建的，则可能高置信度成立。

### 2.1.2. Markov 模糊集合

Markov 不等式为一个非负单变量随机变量 $Z$ 提供了一个上界，该随机变量的均值为 $\mu \geq 0$，且其超过一个正阈值 $\tau>0$ 的概率。正式地，它表明对于模糊集合 $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}\left(\mathbb{R}_{+}\right): \mathbb{E}_{\mathbb{P}}[Z]=\mu\right\}$ 中的每个可能的 $Z$ 的概率分布，$\mathbb{P}(Z \geq \tau) \leq \mu / \tau$。如果 $\mu \leq \tau$，那么 Markov 不等式是紧的，即存在一个概率分布 $\mathbb{P}^{\star} \in \mathcal{P}$，使得该不等式成立为等式。实际上，分布 $\mathbb{P}^{\star}=(1-\mu / \tau) \delta_0 + \mu / \tau \delta_\tau$，其中 $\delta_z$ 是 Dirac 分布，它将点质量放置在 $z \in \mathbb{R}$ 上，属于 $\mathcal{P}$ 并满足 $\mathbb{P}(Z \geq \tau) = \mu / \tau$。这些洞察表明 $\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{P}(Z \geq \tau) = \mu / \tau$，并且当 $\mu \leq \tau$ 时，最大值由 $\mathbb{P}^{\star}$ 达到。因此，Markov 上界可以被解释为一个 DRO 问题的最优值。因此，通常将 $\mathcal{P}$ 称为 Markov 模糊集合。更一般地，我们将与闭合支持集 $\mathcal{Z} \subseteq \mathbb{R}^d$ 和均值向量 $\mu \in \mathbb{R}^d$ 对应的 Markov 模糊集合定义为形式为

$$
\mathcal{P}=\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathrm{P}}[Z]=\mu\right\} .
$$

因此，Markov 模糊集合（2.2）包含所有支持在 $\mathcal{Z}$ 上的分布，这些分布具有相同的均值向量 $\mu$。然而，这些分布可能具有截然不同的形状和高阶矩。Markov 模糊集合上的最坏情况期望有时被用作在随机规划中对期望成本函数的有效计算上界。如果成本函数在不确定问题参数中是凹的，则这些最坏情况期望与 Jensen 不等式紧密相关（Jensen 1906）；详见第 6.1 节。另一方面，如果成本函数是凸的，并且 $\mathcal{Z}$ 是一个多面体，则这些最坏情况期望与 Edmundson-Madansky 不等式相关（Edmundson 1956，Madansky 1959）；详见第 6.2 节。



### 2.1.3. Chebyshev 模糊集合

Chebyshev 不等式提供了一个上界，说明一个具有有限均值 $\mu \in \mathbb{R}$ 和方差 $\sigma^2>0$ 的单变量随机变量 $Z$ 偏离其均值超过 $k>0$ 个标准差的概率。正式地，它表明对于模糊集合 $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathbb{R}): \mathbb{E}_{\mathbb{P}}[Z]=\mu, \mathbb{E}_{\mathbb{P}}\left[Z^2\right]=\sigma^2+\mu^2\right\}$ 中的每个可能的 $Z$ 的概率分布，$\mathbb{P}(|Z-\mu| \geq k \sigma) \leq 1 / k^2$。如果 $k \geq 1$，则 Chebyshev 不等式是紧的。实际上，可以很容易验证分布

$$
\mathbb{P}^{\star}=\frac{1}{2 k^2} \delta_{\mu-k \sigma}+\left(1-\frac{1}{k^2}\right) \delta_\mu+\frac{1}{2 k^2} \delta_{\mu+k \sigma}
$$

是 $\mathcal{P}$ 的一个元素，并且满足 $\mathbb{P}(|Z-\mu| \geq k \sigma)=1 / k^2$。这些洞察表明，Chebyshev 上界可以被解释为一个 DRO 问题的最优值。因此，通常将 $\mathcal{P}$ 称为 Chebyshev 模糊集合。更一般地，我们将与闭合支持集 $\mathcal{Z} \subseteq \mathbb{R}^d$、均值向量 $\mu \in \mathbb{R}^d$ 和二阶矩阵 $M \in \mathbb{S}_{+}^d, M \geq \mu \mu^{\top}$ 对应的 Chebyshev 模糊集合定义为

$$
\mathcal{P}=\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{E}_{\mathrm{P}}[Z]=\mu, \mathbb{E}_{\mathrm{P}}\left[Z Z^{\top}\right]=M\right\}
$$

因此，Chebyshev 模糊集合（2.3）包含所有支持在 $\mathcal{Z}$ 上的分布，这些分布具有相同的均值向量 $\mu$ 和二阶矩阵 $M$（因此也具有相同的协方差矩阵 $\Sigma=M-\mu \mu^{\top} \in \mathbb{S}_{+}^d$）。然而，这些分布可能具有截然不同的形状和高阶矩。

Chebyshev 模糊集合（2.3）捕捉了与多变量 Chebyshev 不等式相关的分布信息（Lal 1955，Marshall 和 Olkin 1960，Tong 1980，Rujeerapaiboon, Kuhn 和 Wiesemann 2018）。在运筹学中，Chebyshev 模糊集合自 Scarf（1958）关于分布鲁棒新闻供应商的开创性工作以来被广泛使用，该工作通常被认为是 DRO 的第一篇论文。从那时起，许多与 Chebyshev 模糊集合相关的 DRO 模型在新闻供应商和投资组合选择问题的背景下涌现。这些模型涉及各种不同的决策标准，如期望值（Gallego 和 Moon 1993，Natarajan 和 Linyi 2007，Popescu 2007），风险价值（El Ghaoui 等 2003，Xu, Caramanis 和 Mannor 2012b，Zymler, Kuhn 和 Rustem 2013a,b，Rujeerapaiboon, Kuhn 和 Wiesemann 2016，Yang 和 Xu 2016，Zhang, Jiang 和 Shen 2018），条件风险价值（Natarajan, Sim 和 Uichanco 2010，Chen, He 和 Zhang 2011，Zymler 等 2013b，Hanasusanto, Kuhn, Wallace 和 Zymler 2015a），谱风险度量（Li 2018）和失真风险度量（Cai, Li 和 Mao 2023，Pesenti, Wang 和 Wang 2024），以及最大最小遗憾标准（Yue, Chen 和 Wang 2006，Perakis 和 Roels 2008）。2008)。除此之外，切比雪夫模糊集在期权和股票定价（Bertsimas 和 Popescu 2002）、统计学和机器学习（Lanckriet, El Ghaoui, Bhattacharyya 和 Jordan 2001, 2002, Strohmann 和 Grudic 2002, Huang, Yang, King, Lyu 和 Chan 2004, Bhattacharyya 2004, Farnia 和 Tse 2016, Nguyen, Shafieezadeh-Abadeh, Yue, Kuhn 和 Wiesemann 2019, Rontsis, Osborne 和 Goulart 2020）、随机规划（Birge 和 Wets 1986, Dulá 和 Murthy 1992, Dokov 和 Morton 2005, Bertsimas, Doan, Natarajan 和 Teo 2010, Natarajan, Teo 和 Zheng 2011）、控制（Van Parys, Kuhn, Goulart 和 Morari 2015, Yang 2018, Xin 和 Goldberg 2021, 2022）、电力系统运行（Xie 和 Ahmed 2017, Zhao 和 Jiang 2017）、复杂网络分析（Van Leeuwaarden 和 Stegehuis 2021, Brugman, Van Leeuwaarden 和 Stegehuis 2022）、排队系统（van Eekelen, Hanasusanto, Hasenbein 和 van Leeuwaarden 2023）、医疗保健（Mak, Rong 和 Zhang 2015, Shehadeh, Cohn 和 Jiang 2020）、极端事件分析（Lam 和 Mottet 2017）等方面都得到了广泛应用。


2.1.4. 切比雪夫模糊集与不确定的矩

在已知 $\mathbb{P}$ 的一阶和二阶矩时，使用切比雪夫模糊集是合适的，但所有更高阶的矩是未知的。然而，在实际中，即使是已知的一阶和二阶矩也从来不能绝对确定。相反，它们必须从统计数据中估计，因此会受到估计误差的影响。这促使 El Ghaoui 等人（2003）引入了具有不确定矩的切比雪夫模糊集，可以表示为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}):\left(\mathrm{E}_{\mathrm{P}}[Z], \mathrm{B}_{\mathrm{P}}\left[Z Z^{\top}\right]\right) \in \mathcal{F}\right\} .
$$

这里，$\mathcal{F} \subseteq \mathbb{R}^d \times \mathbb{S}_{+}^d$ 是一个凸集，用于捕捉矩的不确定性。显然，$\mathcal{P}$ 可以表示为一系列切比雪夫模糊集的并集，即：

$$
\mathcal{P}=\bigcup_{(\mu, M) \in \mathcal{F}}\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{E}_{\mathrm{P}}[Z]=\mu, \mathbb{E}_{\mathrm{P}}\left[Z Z^{\top}\right]=M\right\} .
$$

注意，具有不确定矩的切比雪夫模糊集涵盖了支持集模糊集、马尔可夫模糊集和切比雪夫模糊集作为特例。它们通过设置 $\mathcal{F}=\mathbb{R}^d \times \mathbb{S}_{+}^d, \mathcal{F}=\{\mu\} \times \mathbb{S}_{+}^d$ 和 $\mathcal{F}=\{\mu\} \times\{M\}$ 分别恢复。

El Ghaoui 等人（2003）通过以下盒子捕捉矩的不确定性：

$$
\mathcal{F}=\left\{(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \underline{\mu} \leq \mu \leq \bar{\mu}, \underline{M} \leq M \leq \bar{M}\right\}
$$

该框由矩的界限参数化，其中 $\underline{\mu}, \bar{\mu} \in \mathbb{R}^d$ 和 $\underline{M}, \bar{M} \in \mathbb{S}_{+}^d$。

给定对未知均值向量和协方差矩阵的噪声估计 $\hat{\mu}$ 和 $\hat{\Sigma}$，Delage 和 Ye（2010）提出了以下模糊集：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \begin{array}{l}
\left(\mathbb{E}_{\mathrm{P}}[Z]-\hat{\mu}\right)^{\top} \hat{\Sigma}^{-1}\left(\mathbb{E}_{\mathrm{P}}[Z]-\hat{\mu}\right) \leq \gamma_1 \\
\mathbb{E}_{\mathrm{P}}\left[(Z-\hat{\mu})(Z-\hat{\mu})^{\top}\right] \leq \gamma_2 \hat{\Sigma}
\end{array}\right\} .
$$

通过构造，$\mathcal{P}$ 包含所有分布在 $\mathcal{Z}$ 上，其一阶矩位于以 $\hat{\mu}$ 为中心的椭球内，二阶矩（相对于 $\hat{\mu}$）位于以 $\gamma_2 \hat{\Sigma}$ 为顶点的半正定锥内。一个基本的计算揭示了：

$$
\mathbb{E}_{\mathrm{P}}\left[(Z-\hat{\mu})(Z-\hat{\mu})^{\top}\right]=\mathrm{E}_{\mathrm{P}}\left[Z Z^{\top}\right]-\mathbb{B}_{\mathrm{P}}[Z] \hat{\mu}^{\top}-\hat{\mu} \mathrm{E}_{\mathrm{P}}[Z]^{\top}+\hat{\mu} \hat{\mu}^{\top}
$$
因此，$\mathbb{P}$ 可以视为一个具有不确定矩的切比雪夫模糊集合。实际上，如果我们将矩不确定集定义为

$$\mathcal{F}=\left\{(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \begin{array}{l}(\mu-\hat{\mu})^{\top} \hat{\Sigma}(\mu-\hat{\mu}) \leq \gamma_1 \\ M-\mu \hat{\mu}^{\top}-\hat{\mu} \mu^{\top}+\hat{\mu} \hat{\mu}^{\top} \leq \gamma_2 \hat{\Sigma}\end{array}\right\}.
$$

Delage 和 Ye（2010）表明，如果 $\rho$ 和 $\sum$ 被设置为样本均值和从有限数量的独立样本中提取的协方差矩阵，那么可以调整集参数 $\gamma_1 \geq 0$ 和 $\gamma_2 \geq 1$，以确保 $\mathbb{P}$ 属于 $\mathcal{P}$，并具有任何期望的置信度。

具有不确定矩的切比雪夫和马尔可夫模糊集合在多个领域找到了应用，涉及的领域包括控制（Naka, Jiang 和 Shen 2021）、积分随机规划（Bertamas, Natarajan 和 Teo 2004）、事件分析（Bai, Lam 和 Doung 2023）、机制设计和定价（Bertsimas 2024）等众多领域。

不确定集 $\mathcal{F}$ 对于 $\mathbb{P}$ 的一阶和二阶矩通常涉及一些差异性。例如，矩阵范数，如 Frobenius 范数、谱范数或核范数（Bertand 2009, 49）提供了自然的度量来量化协方差矩阵的不同。差异性量化了正态分布 $N(\mu, \Sigma)$ 和 $N(\mu, 2)$ 之间的差异，使用的是概率度量或信息理论的散度，如 Kullback-Leibler 散度（Kullback 和 Leibler 1951）或其他谱散度（Zhang 2014）。

正如我们在第23节中将更详细地讨论的那样，两个正态分布 $N(\mu, \Sigma)$ 和 $\mathcal{N}(\mu, \mathbf{2})$ 之间的 2 Wasserstein 距离与 Gelbrich 距离一致，后者是在基础均值-协方差对 $(\mu, \Sigma)$ 和 $(\mu, 2)$ 之间的距离。接下来，我们首先提供 Gelbrich 距离的正式定义，然后举例说明如何利用它来定义一个矩不确定集 $\mathcal{F}$。

定义 2.1 (Gelbrich 距离)。在 $\mathbb{R}^d \times \mathbb{S}_{+}^d$ 中，两个均值协方差对 $(\mu, \Sigma)$ 和 $(\hat{\mu}, \hat{\Sigma})$ 之间的 Gelbrich 距离定义为

$$
\mathrm{G}((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma}))=\sqrt{\|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}\left(\Sigma+\hat{\Sigma}-2\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right)} .
$$

Gelbrich 距离是非负的、对称的和次可加的，且当且仅当 $(\mu, \Sigma) = (\hat{\mu}, \hat{\Sigma})$ 时它为零。因此，它表示了 $\mathbb{R}^d \times \mathbb{S}_{+}^d$ 上的一个度量（Givens 和 Short 1984, 第239页）。当 $\mu = \hat{\mu}$ 时，Gelbrich 距离退化为 $\Sigma$ 和 $\hat{\Sigma}$ 之间的 Bures 距离，后者作为量子信息理论中密度矩阵之间相异性的度量。Bures 距离已知能在正半定矩阵空间上诱导一个黎曼度量（Bhatia, Jain 和 Lim 2018, 2019）。当 $\Sigma$ 和 $\hat{\Sigma}$ 可以同时对角化时，它们的 Bures 距离与它们谱之间的 Hellinger 距离一致。Hellinger 距离与信息理论中广泛使用的 Fisher-Rao 度量密切相关（Liese 和 Vajda 1987）。尽管 Gelbrich 距离是非凸的，但平方的 Gelbrich 距离在其两个参数中是联合凸的。这是以下命题的直接结果，该命题可以在（Olkin 和 Pukelsheim 1982, Dowson 和 Landau 1982, Givens 和 Shortt 1984, Panaretos 和 Zemel 2020）中找到。

命题 2.2 (Gelbrich 距离的 SDP 表示)。对于任意均值协方差对 $(\mu, \Sigma)$ 和 $(\hat{\mu}, \hat{\Sigma})$ 在 $\mathbb{R}^d \times \mathbb{S}_{+}^d$ 中，我们有

$$
\mathrm{G}^2((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma}))=\left\{\begin{array}{cl}
\min _{C \in \mathbb{R}^{d \times d}} & \|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}(\Sigma+\hat{\Sigma}-2 C) \\
\text { s.t. } & {\left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 .}
\end{array}\right.
$$

证明：在证明过程中，我们保持 $\mu, \hat{\mu}$ 和 $\Sigma$ 不变，并将 $\hat{\Sigma}$ 视为参数。我们还使用 $f(\hat{\Sigma})$ 作为 (2.5) 左侧的简写，$g(\hat{\Sigma})$ 作为 (2.5) 右侧的简写。基础操作显示

$$
g(\hat{\Sigma})=\|\mu-\hat{\mu}\|_2^2+\operatorname{Tr}(\Sigma+\hat{\Sigma})-\left\{\begin{array}{cl}
\max _{C \in \mathbb{R}^{d \times d}} & \operatorname{Tr}(2 C) \\
\text { s.t. } & {\left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 .}
\end{array}\right.
$$

(2.6) 中的最大化问题是以下最小化问题的对偶问题。

$$
\begin{array}{cl}
\inf _{A_{11}, A_{22} \in \mathbb{S}^d} & \operatorname{Tr}\left(A_{11} \Sigma\right)+\operatorname{Tr}\left(A_{22} \hat{\Sigma}\right) \\
\text { s.t. } & {\left[\begin{array}{cc}
A_{11} & I_d \\
I_d & A_{22}
\end{array}\right] \geq 0}
\end{array}
$$

由于 $A_{11}=A_{22}=2 I_d$ 构成了对偶问题的 Slater 点（Ben-Tal 和 Nemirovski 2001, Theorem 2.4.1），因此强对偶性成立。Slater 点的存在进一步意味着 (2.6) 中的原始最大化问题以及 (2.5) 中的最小化问题是可解的。根据 (Bernstein 2009, Corollary 8.2.2)，为了保证对偶可行性，$A_{11}$ 和 $A_{22}$ 必须是正定的。因此，它们是可逆的。我们可以使用 Schur 互补论证（Ben-Tal 和 Nemirovski 2001, Lemma 4.2.1）将对偶问题简化为

$$
\inf _{A_{11} \geq A_{22}^{-1}>0} \operatorname{Tr}\left(A_{11} \Sigma\right)+\operatorname{Tr}\left(A_{22} \hat{\Sigma}\right)=\inf _{A_{22}>0} \operatorname{Tr}\left(A_{22}^{-1} \Sigma\right)+\operatorname{Tr}\left(A_{22} \hat{\Sigma}\right),
$$
其中等式成立是因为 $\Sigma \geq 0$。得到的最小化问题的最优值在 $\hat{\Sigma}$ 中是凹的，并且是上半连续的，因为它构成了 $\hat{\Sigma}$ 的仿射函数的逐点下确界。因此，$g(\hat{\Sigma})$ 是凸的并且下半连续的。现在我们展示，如果 $\hat{\Sigma} > 0$，那么 (2.7) 中对 $A_{22}$ 的凸最小化问题可以解析求解。为此，我们构造一个满足问题一阶最优性条件的正定矩阵 $A_{22}^{\star}$：

$$
\hat{\Sigma}-A_{22}^{-1} \Sigma A_{22}^{-1}=0 \quad \Longleftrightarrow \quad A_{22} \hat{\Sigma} A_{22}-\Sigma=0 .
$$

事实上，右侧的二次方程两边同时与 $\hat{\Sigma}^{\frac{1}{2}}$ 相乘，得到等价方程 $\left(\hat{\Sigma}^{\frac{1}{2}} A_{22} \hat{\Sigma}^{\frac{1}{2}}\right)^2=\hat{\Sigma}^{\frac{1}{2}} \Sigma^{\frac{1}{2}}$。由于 $\hat{\Sigma} > 0$，该方程唯一解为 $A_{22}^{\star}=\hat{\Sigma}^{-\frac{1}{2}}\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}} \hat{\Sigma}^{-\frac{1}{2}}$。将 $A_{22}^{\star}$ 代入 (2.7) 中，结果显示对偶最小化问题的最优值为 $2 \operatorname{Tr}\left(\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right)$。然后将该值代入 (2.6) 中，显示当 $\hat{\Sigma} > 0$ 时，$g(\hat{\Sigma})=f(\hat{\Sigma})$。

接下来我们展示，如果 $\hat{\Sigma}$ 是奇异的，那么 $g(\hat{\Sigma}) = f(\hat{\Sigma})$。为此，我们回顾 (Nguyen, Shafieezadeh-Abadeh, Kuhn 和 Mohajerin Esfahani 2023, Lemma A.2) 中的结论，即矩阵平方根在 $\mathbf{S}_{+}^d$ 上是连续的，这意味着 $f(\hat{\Sigma})$ 在 $\mathbb{S}_{+}^d$ 上是连续的。对于任何奇异的 $\hat{\Sigma} \geq 0$，我们有

$$
f(\hat{\mathbf{\Sigma}})=\liminf _{\hat{\mathbf{\Sigma}}^{\prime} \rightarrow \hat{\mathbf{\Sigma}}, \hat{\mathbf{\Sigma}}^{\prime}>0} f\left(\hat{\mathbf{\Sigma}}^{\prime}\right)=\liminf _{\hat{\mathbf{\Sigma}}^{\prime} \rightarrow \hat{\mathbf{\Sigma}}, \hat{\Sigma}^{\prime}>0} g\left(\hat{\Sigma}^{\prime}\right)=g(\hat{\mathbf{\Sigma}}) .
$$

这里，第一个等式利用了 $f$ 的连续性，第二个等式成立是因为对于每个 $\hat{\Sigma}^{\prime} > 0$，$f\left(\hat{\Sigma}^{\prime}\right)=g\left(\hat{\Sigma}^{\prime}\right)$。第三个等式来自于 $g$ 的凸性和下半连续性，这意味着极限下确界不能小于或大于 $g(\hat{\Sigma})$，分别地。由此，证明完毕。

命题 2.2 显示，平方的 Gelbrich 距离与一个可解的半正定程序的最优值相同。这使得 Gelbrich 距离在计算中具有吸引力。作为副产品，命题 2.2 的证明揭示了平方 Gelbrich 距离在其定义域上是凸的并且连续的。

根据 Nguyen、Shafiee、Filipovic 和 Kuhn (2021) 的方法，我们现在可以将 Gelbrich 模糊集合作为不确定矩的 Chebyshev 模糊集合 (2.4) 的一个实例。相应的矩不确定集合如下所示：

$$
\mathcal{F}=\left\{(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \begin{array}{l}
\exists \Sigma \in \mathbb{S}_{+}^d \text { with } M=\Sigma+\mu \mu^{\top}, \\
\mathrm{G}((\mu, \Sigma),(\hat{\mu}, \hat{\Sigma})) \leq r
\end{array}\right\},
$$

其中 $(\hat{\mu}, \hat{\Sigma})$ 是一个名义均值-协方差对，半径 $r \geq 0$ 是一个可调节的大小参数。下面我们将 $\mathcal{F}$ 称为 Gelbrich 不确定集合。下一个命题建立了 $\mathcal{F}$ 的基本拓扑和计算性质。

命题 2.3 (Gelbrich 不确定集合)。不确定集合 $\mathcal{F}$ 如 (2.8) 所定义，是凸的并且是紧的。此外，它具有以下半正定表示：

$$
\mathcal{F}=\left\{\begin{array}{ll}
(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: & \exists C \in \mathbb{R}^{d \times d}, U \in \mathbb{S}_{+}^d \text { with } \\
\|\hat{\mu}\|_2^2-2 \mu^{\top} \hat{\mu}+\operatorname{Tr}(M+\hat{\Sigma}-2 C) \leq r^2, \\
{\left[\begin{array}{cc}
M-U & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0,\left[\begin{array}{cc}
U & \mu \\
\mu^{\top} & 1
\end{array}\right] \geq 0}
\end{array}\right\} .
$$

证明。该证明利用了命题 2.2 中建立的平方 Gelbrich 距离的半正定表示。首先注意到，如果 $M = \Sigma + \mu \mu^{\top}$，那么

$$
\|\mu - \hat{\mu}\|_2^2 + \operatorname{Tr}(\Sigma + \hat{\Sigma} - 2 C) = \|\hat{\mu}\|_2^2 - 2 \mu^{\top} \hat{\mu} + \operatorname{Tr}(M + \hat{\Sigma} - 2 C) .
$$

根据命题 2.2，Gelbrich 不确定集合 $\mathcal{F}$ 可以表示为

$$
\mathcal{F} = \left\{ \begin{array}{ll}
& \exists C \in \mathbb{R}^{d \times d} \text{ with } \\
& \|\hat{\mu}\|_2^2 - 2 \mu^{\top} \hat{\mu} + \operatorname{Tr}(M + \hat{\Sigma} - 2 C) \leq r^2, \\
{\left[\begin{array}{cc}
M - \mu \mu^{\top} & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0}
\end{array} \right\} .
$$

一个标准的 Schur 余补推论进一步揭示

$$
\left[\begin{array}{cc}
M - \mu \mu^{\top} & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 \Longleftrightarrow \exists U \in \mathbb{S}_{+}^d \text{ with } \left[\begin{array}{cc}
M - U & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0, \left[\begin{array}{cc}
U & \mu \\
\mu^{\top} & 1
\end{array}\right] \geq 0 .
$$

因此，Gelbrich 不确定集合 $\mathcal{F}$ 具有命题中给出的半正定表示。由于该表示通过锥不等式定义的集合的投影表达了 $\mathcal{F}$，因此其凸性显而易见。

接下来需要证明的是，$\mathcal{F}$ 是紧的。为此，我们定义

$$
\mathcal{V} = \left\{ (\mu, \Sigma) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \mathrm{G}((\mu, \Sigma), (\hat{\mu}, \hat{\Sigma})) \leq r \right\}
$$

作为关于 $(\hat{\mu}, \hat{\Sigma})$ 的半径为 $r$ 的球，使用 Gelbrich 距离来度量。注意到 $\mathcal{F} = f(\mathcal{V})$，其中变换 $f: \mathbb{R}^d \times \mathbb{S}_{+}^d \rightarrow \mathbb{R}^d \times \mathbb{S}_{+}^d$ 定义为 $f(\mu, \Sigma) = \left(\mu, \Sigma + \mu \mu^{\top}\right)$。现在我们将证明 $\mathcal{V}$ 是紧的。由于 $f$ 是连续的，并且紧性在连续变换下保持，因此这将直接推导出 $\mathcal{F}$ 是紧的。显然，$\mathcal{V}$ 是闭的，因为 Gelbrich 距离是连续的。为了证明 $\mathcal{V}$ 也是有界的，固定任意 $(\mu, \Sigma) \in \mathcal{V}$。根据 Gelbrich 距离的定义，我们有 $\|\mu - \hat{\mu}\| \leq r^2$。此外，我们可以得到

$$
\begin{aligned}
\operatorname{Tr}\left(\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right) &= \max_{C \in \mathbb{R}^{d \times d}} \left\{ \operatorname{Tr}(C): \left[\begin{array}{cc}
\Sigma & C \\
C^{\top} & \hat{\Sigma}
\end{array}\right] \geq 0 \right\} \\
&\leq \max_{C \in \mathbb{R}^{d \times d}} \left\{ \operatorname{Tr}(C): C_{ij}^2 \leq \Sigma_{ii} \hat{\Sigma}_{jj} \ \forall i,j \in[d] \right\} \leq \sqrt{\operatorname{Tr}(\Sigma) \operatorname{Tr}(\hat{\Sigma})},
\end{aligned}
$$

其中，等式已在命题 2.2 的证明中建立。两个不等式来自于线性矩阵不等式的放松，它利用了一个观察结果，即一个正半定矩阵的所有二阶主子式是非负的，并且使用了 Cauchy-Schwarz 不等式。因此，$\Sigma$ 满足

$$
\left.\left(\operatorname{Tr}(\Sigma)^{\frac{1}{2}} - \operatorname{Tr}(\hat{\Sigma})^{\frac{1}{2}}\right)^2 \leq \operatorname{Tr}\left(\Sigma + \hat{\Sigma} - 2\left(\hat{\Sigma}^{\frac{1}{2}} \Sigma \hat{\Sigma}^{\frac{1}{2}}\right)^{\frac{1}{2}}\right)\right) \leq r^2,
$$

其中第二个不等式成立是因为 $(\mu, \Sigma) \in \mathcal{V}$。因此，我们可以得出结论：$\operatorname{Tr}(\Sigma) \leq \left(r + \operatorname{Tr}(\hat{\Sigma})^{\frac{1}{2}}\right)^2$，进而得到 $0 \leq \Sigma \leq \left(r + \left(\operatorname{Tr}(\hat{\Sigma})\right)^{\frac{1}{2}}\right)^2 I_d$。总之，我们已经证明了 $\mu$ 和 $\Sigma$ 都属于有界集合。由于选择的 $(\mu, \Sigma) \in \mathcal{V}$ 是任意的，这证明了 $\mathcal{V}$ 确实是有界的，因此是紧的。得证。

命题 2.2 显示了不确定性集合 $\mathcal{F}$ 是凸的。这一点令人惊讶，因为 $\mathcal{F} = f(\mathcal{V})$，其中 Gelbrich 球 $\mathcal{V}$ 在均值-协方差对的空间中是凸的，得益于命题 2.2，而 $f$ 是一个二次双射。事实上，通常只有在仿射变换下，凸性才会被保留。

Gelbrich 不确定性集是由 Nguyen 等人（2021）在稳健投资组合优化的背景下引入的。它们还在机器学习（Bui, Nguyen 和 Nguyen 2022, Vu, Tran, Yue 和 Nguyen 2021, Nguyen, Bui 和 Nguyen 2022a）、估计（Nguyen 等人 2023）、滤波（Shafieezadeh-Abadeh, Nguyen, Kuhn 和 Mohajerin Esfahani 2018, Kargin, Hajar, Malik 和 Hassibi 2024b）以及控制（McAllister 和 Mohajerin Esfahani 2023, Al Taha, Yan 和 Bitar 2023, Hajar, Kargin 和 Hassibi 2023, Hakobyan 和 Yang 2024, Taşkesen, Iancu, Koçyiğit 和 Kuhn 2024, Kargin, Hajar, Malik 和 Hassibi 2024a,c,d）中找到了应用。



### 2.1.5. 均值-离散不确定性集

如果 $\mathcal{K} \subseteq \mathbb{R}^k$ 是一个适当的凸锥，且 $v_1, v_2 \in \mathbb{R}^k$，则不等式 $v_1 \leq \mathcal{K} v_2$ 表示 $v_2 - v_1 \in \mathcal{K}$。此外，函数 $G: \mathbb{R}^m \rightarrow \mathbb{R}^k$ 被称为 $\mathcal{K}$-凸的，如果满足

$$
G\left(\theta v_1 + (1 - \theta) v_2\right) \leq \mathcal{K} \theta G\left(v_1\right) + (1 - \theta) G\left(v_2\right) \quad \forall v_1, v_2 \in \mathbb{R}^m, \forall \theta \in [0,1] .
$$

对应于一个凸闭支持集 $\mathcal{Z} \subseteq \mathbb{R}^d$，均值向量 $\mu \in \mathbb{R}^d$，一个 $\mathcal{K}$-凸的离散函数 $G: \mathbb{R}^m \rightarrow \mathbb{R}^k$ 和一个离散界限 $g \in \mathbb{R}^k$，均值-离散不确定性集定义为

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{B}_{\mathrm{P}}[Z] = \mu, \mathbb{E}_{\mathrm{P}}[G(Z)] \leq \mathcal{K} g\right\}
$$

均值-离散不确定性集具有很强的表达能力，即它能够建模未知概率分布的各种典型特征。例如，如果 $\|\cdot\|$ 是 $\mathbb{R}^d$ 上的一个范数，$G(z) = \|z - \mu\|$ 是通常意义上的凸函数，且 $g = \sigma \in \mathbb{R}_{+}$，则所有的分布 $\mathbb{P} \in \mathcal{P}$ 的均值绝对偏差都被 $\sigma$ 有界。或者，如果 $G(z) = (z - \mu)(z - \mu)^{\top}$ 是 $\mathbb{S}_{+}^d$-凸的，且 $g = \Sigma \in \mathbb{S}_{+}^d$，则 $\mathcal{P}$ 退化为具有时刻不确定性的 Chebyshev 不确定性集。具体来说，任何 $\mathbb{P} \in \mathcal{P}$ 的协方差矩阵都被 $\Sigma$ 限制在 Loewner 顺序中。Wiesemann, Kuhn 和 Sim (2014) 表明，包含 $d$ 维随机向量 $Z$ 分布的不确定性集 $\mathcal{P}$ 与提升的不确定性集紧密相关：

$$
\mathcal{Q} = \left\{\mathbb{Q} \in \mathcal{P}(\mathcal{C}): \mathbb{B}_{\mathrm{Q}}[Z] = \mu, \mathbb{E}_{\mathrm{Q}}[U] = g\right\}
$$

其中支持集 $\mathcal{C} = \left\{(z, u) \in \mathcal{Z} \times \mathbb{R}^k: G(z) \leq_{\mathcal{K}} u\right\}$，包含 $Z$ 和一个辅助的 $m$ 维随机向量 $U$ 的联合分布。实际上，可以证明 $\mathcal{P} = \left\{Q_z: Q \in \mathcal{Q}\right\}$，其中 $Q_z$ 表示 $Q$ 下的 $Z$ 的边际分布。由于损失函数仅依赖于 $Z$ 而不依赖于 $U$，这一推理意味着（1.2）中的内层最坏情况期望问题满足

$$
\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{B}_{\mathrm{P}}[\ell(x, Z)] = \sup _{\mathrm{Q} \in \mathcal{Q}} \mathbb{E}_{\mathrm{Q}}[\ell(x, Z)] .
$$

因此，可以用提升的不确定性集 $\mathcal{Q}$ 替代原始的不确定性集 $\mathcal{P}$。这是有用的，因为 $\mathcal{Q}$ 是一个简单的 Markov 不确定性集，只指定联合随机向量 $(Z, U)$ 的支持集 $\mathcal{C}$ 和均值 $(\mu, g)$。此外，可以证明 $\mathcal{Z}$ 是凸的，因为 $\mathcal{Z}$ 本身是凸的，且 $G$ 是 $\mathcal{K}$-凸的。总之，具有均值-离散不确定性集（形式为（2.9））的 DRO 问题可以系统地转化为具有 Markov 不确定性集的 DRO 问题。

一种更一般的均值-离散不确定性集可以用来塑造 $Z$ 在 $\mathbb{P}$ 下的矩生成函数。具体来说，Chen、Sim 和 Xu（2019）引入了熵支配不确定性集：

$$
\mathcal{P} = \left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}[Z] = \mu, \log \left(\mathbb{E}_{\mathbb{P}}\left[\exp \left(\theta^{\top}(Z-\mu)\right)\right]\right) \leq g(\theta) \forall \theta \in \mathbb{R}^d \right\},
$$

其中 $g: \mathbb{R}^d \rightarrow \mathbb{R}$ 是一个凸且连续二次可微的函数，满足 $g(0) = 0$ 且 $\nabla g(0) = 0$。由 $\theta$ 参数化的约束对聚合生成函数（即对中心化随机变量 $Z-\mu$ 的对数矩生成函数）施加了连续的上界。$g$ 的选择决定了不确定性集中包含的具体分布类别。例如，如果 $g(\theta) = \sigma^2 \theta^{\top} \theta / 2$（其中 $\sigma > 0$），则该不确定性集只包含方差代理为 $\sigma^2$ 的亚高斯分布。亚高斯分布是尾部受高斯分布尾部限制的概率分布。它们在概率论和统计学中扮演着重要角色，特别是在浓度不等式和高维现象的研究中（Vershynin 2018，Wainwright 2019）。

熵支配不确定性集对 $\mathbb{P}$ 强加了无限多的约束。Chen 等人（2019）表明，在该不确定性集上的最坏情况期望问题可以重新表述为半无限锥程序。他们提出了一种切平面算法，以高效地求解这些锥程序。熵支配不确定性集还在非线性和 PDE 约束的 DRO 问题研究中得到了应用（Milz 和 Ulbrich 2020，2022）。Chen、Fu、Si、Sim 和 Xiong（2023）考虑了广义的熵支配不确定性集。


### 2.1.6. Higher-order Moment Ambiguity Sets

Markov 和 Chebyshev 不确定性集仅对 $\mathbb{P}$ 的一阶和/或二阶矩施加条件。具有这种不确定性集的 DRO 问题通常是可处理的。在本节中，我们简要讨论对 $\mathbb{P}$ 的高阶（多项式）矩施加条件的矩不确定性集，这通常会导致 NP-hard 的 DRO 问题（Popescu 2005，第 4.5 和 4.6 命题）。

现在假设 $\mathcal{Z}$ 是一个闭半代数集，可以表示为有限多个多项式不等式的可行集。此外，定义在 $z \in \mathbb{R}^d$ 中的阶数为 $\alpha \in \mathbb{Z}_{+}^d$ 的单项式为函数 $\prod_{i=1}^d z_i^{\alpha_i}$，我们简洁地表示为 $z^\alpha$。由有限索引集 $\mathcal{A} \subseteq \mathbb{Z}_{+}^d$ 和矩界限 $m_\alpha \in \mathbb{R}, \alpha \in \mathcal{A}$ 诱导的高阶矩不确定性集为

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathbb{P}}\left[Z^\alpha\right] \leq m_\alpha \forall \alpha \in \mathcal{A}\right\}
$$

因此，评估多项式函数（或半代数集的特征函数）在所有分布 $\mathcal{P}$ 上的最坏情况期望就等同于求解一个广义的矩问题。这个矩问题及其对偶问题构成了半无限线性规划，可以被重新构造为某些矩锥和相应的非负多项式对偶锥上的有限维锥优化问题（Karlin 和 Studden 1966，Zuluaga 和 Pena 2005）。尽管一般情况下是 NP-hard，这些锥优化问题可以通过多项式优化工具（Parrilo 2000, 2003，Lasserre 2001, 2009）逼近为越来越紧的可处理半定规划序列。这种通用技术为不确定性集 $\mathcal{P}$ 提供了最坏情况期望界限和广义 Chebyshev 不等式（Bertsimas 和 Sethuraman 2000，Lasserre 2002，Popescu 2005，Lasserre 2008）。此外，它还为最坏情况风险度量提供了紧的界限（Natarajan，Pachamanova 和 Sim 2009a）。


## 2.2. $\phi$-Divergence Ambiguity Sets

概率分布之间的差异通常通过 $\phi$-散度来量化，这由一个熵函数 $\phi$ 唯一决定。

**定义 2.4** (熵函数)。熵函数 $\phi: \mathbb{R} \rightarrow \overline{\mathbb{R}}$ 是一个下半连续凸函数，满足 $\phi(1)=0$ 且 $\phi(s)=+\infty$ 对于所有 $s<0$。

注意，任何熵函数 $\phi$ 相对于其定义域是连续的。事实上，这对于任何一元的凸下半连续函数都是成立的。然而，我们强调，多元凸下半连续函数可能在其定义域内有不连续点（Rockafellar 和 Wets 2009, 示例 2.38）。$\phi$-散度的概念依赖于熵函数 $\phi$ 的视角 $\phi^\pi$。

**定义 2.5** ($\phi$-散度 (Csiszár 1963, 1967, Ali 和 Silvey 1966))。$\mathbb{P} \in \mathcal{P}(\mathcal{Z})$ 相对于 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 的（广义）$\phi$-散度定义为

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})=\int_{\mathcal{Z}} \phi^\pi\left(\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \rho}(z), \frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z)\right) \mathrm{d} \rho(z),
$$

其中 $\phi$ 是熵函数，$\rho \in \mathcal{M}_{+}(\mathcal{Z})$ 是任何支配测度。这意味着 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$ 相对于 $\rho$ 是绝对连续的，即 $\mathbb{P}, \hat{\mathbb{P}} \ll \rho$。

根据 $\phi^\pi$ 的定义以及我们的约定 $0 \phi(s / 0)$ 应该解释为 recession function $\phi^{\infty}(s)$，$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$ 可以重新表达为

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})=\int_{\mathcal{Z}} \frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \cdot \phi\left(\frac{\frac{\mathrm{d}}{\mathrm{P}}(z)}{\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z)}\right) \mathrm{d} \rho(z) .
$$

一个支配测度 $\rho$ 总是存在的，但它必须依赖于 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$。例如，可以设置 $\rho = \mathbb{P} + \hat{\mathbb{P}}$。绝对连续性条件 $\mathbb{P} \ll \rho$ 和 $\hat{\mathbb{P}} \ll \rho$ 确保了 Radon-Nikodym 导数 $\frac{\mathrm{d}}{\mathrm{d} \rho}$ 和 $\frac{\mathrm{d}}{\mathrm{d} \rho}$ 分别是良定义的。以下命题推导了一个通用的 $\phi$-散度的对偶表示，揭示了 $\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$ 实际上与选择 $\rho$ 无关。

命题 2.6 （$\phi$-散度的对偶表示）。我们有

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})=\sup _{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \, \mathrm{d}\mathbb{P}(z) - \int_{\mathcal{Z}} \phi^*(f(z)) \, \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \operatorname{dom}\left(\phi^*\right)$ 的集合。

Proof. As the entropy function $\phi(s)$ is proper, convex and lower semicontinuous on $\mathbb{R}$ and as $0 \phi(s / 0)$ is interpreted as the recession function $\phi^{\infty}(s)$, the perspective function $\phi^\pi(s, t)=t \phi(s / t)$ is proper, convex and lower semicontinuous on $\mathbb{R} \times \mathbb{R}_{+}$. By (Rockafellar 1970, Theorem 12.2), $\phi^\pi(s, t)$ can therefore be expressed as the conjugate of its conjugate. Note that the conjugate of $\phi^\pi(s, t)$ satisfies

$$
\begin{aligned}
\left(\phi^\pi\right)^*(f, g) & =\sup _{s \in \mathbb{R}, t \in \mathbb{R}_{+}} f s+g t-t \phi(s / t) \\
& =\sup _{t \in \mathbb{R}_{+}} g t+t \phi^*(f)=\left\{\begin{array}{cl}
0 & \text { if } f \in \operatorname{dom}\left(\phi^*\right) \text { and } g+\phi^*(f) \leq 0, \\
+\infty & \text { otherwise, }
\end{array}\right.
\end{aligned}
$$

for all $f, g \in \mathbb{R}$. The second equality in the above expression follows from (Rockafellar 1970, Theorem 16.1). As $\phi^\pi(s, t)=\sup _{f, g \in \mathbb{R}} s f+t g-\left(\phi^\pi\right)^*(f, g)$ by virtue of (Rockafellar 1970, Theorem 12.2), the $\phi$-divergence is thus given by

$$
\begin{aligned}
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) & =\int_{\mathcal{Z}} \sup _{f, g \in \mathbb{R}}\left\{\frac{\mathrm{~d} \mathbb{P}}{\mathrm{~d} \rho}(z) \cdot f+\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \cdot g-\left(\phi^\pi\right)^*(f, g)\right\} \mathrm{d} \rho(z) \\
& =\int_{\mathcal{Z}} \sup _{f \in \operatorname{dom}\left(\phi^*\right)}\left\{\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \rho}(z) \cdot f-\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \cdot \phi^*(f)\right\} \mathrm{d} \rho(z) \\
& =\sup _{f \in \mathcal{F}} \int_{\mathcal{Z}}\left\{\frac{\mathrm{d} \mathbb{P}}{\mathrm{~d} \rho}(z) \cdot f(z)-\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{~d} \rho}(z) \cdot \phi^*(f(z))\right\} \mathrm{d} \rho(z)
\end{aligned}
$$

第二个等式利用了我们上面推导出的$\left(\phi^\pi\right)^*$的显式公式，而第三个等式则根据（Rockafellar 和 Wets 2009, 定理 14.60）得出。这个定理之所以适用，是因为函数 $h: \operatorname{dom}\left(\phi^*\right) \times \mathcal{Z} \rightarrow \mathbb{R}$ 定义为

$$
h(f, z)=\frac{\mathrm{d} \mathbb{P}}{\mathrm{d} \rho}(z) \cdot f-\frac{\mathrm{d} \hat{\mathbb{P}}}{\mathrm{d} \rho}(z) \cdot \phi^*(f)
$$

在 $f$ 中是连续的，并且在 $z$ 中是 Borel 可测的，因此根据（Rockafellar 和 Wets 2009, 示例 14.29），它构成了一个 Carathéodory 积分器。然后，由于 Radon-Nikodym 导数的定义，该声明立即得出。

命题 2.6 显示了$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$在$\mathbb{P}$和$\hat{\mathbb{P}}$中是联合凸的。如果$\phi(s)$随着$s$的增长速度超过线性，即如果其渐近增长率$\phi^{\infty}(1)$是无穷大的，那么当且仅当$\frac{\mathrm{d}}{\mathrm{d} \rho}(z)=0$对于$\rho$几乎所有的$z \in \mathcal{Z}$，且$\frac{\mathrm{d} \hat{\mathrm{P}}}{\mathrm{d} \rho}(z)=0$时，$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$是有限的。换句话说，$\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})$只有在$\mathbb{P} \ll \hat{\mathbb{P}}$时才是有限的。在这种特殊情况下，Radon-Nikodym 导数的链式法则意味着$\frac{\mathrm{dP}}{\mathrm{d} \rho} / \frac{\mathrm{d} \mathrm{P}}{\mathrm{d} \rho}=\frac{\mathrm{dP}}{\mathrm{dP}}$。如果$\phi^{\infty}(1)=\infty$，则$\phi$-散度有更常见（但较不一般）的表示形式：

$$
\mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}})= \begin{cases}\int_{\mathcal{Z}} \phi\left(\frac{\mathrm{dP}}{\mathrm{~d} \hat{\mathbb{P}}}(z)\right) d \hat{\mathbb{P}}(z) & \text { if } \mathbb{P} \ll \hat{\mathbb{P}} \\ +\infty & \text { otherwise. }\end{cases}
$$

我们现在可以定义$\phi$-散度模糊集为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\} .
$$

该集合包含所有支持在$\mathcal{Z}$上的概率分布$\mathbb{P}$，其相对于某个指定参考分布$\hat{\mathbb{P}}$的$\phi$-散度最大为$r \geq 0$。


**Remark 2.7 (Csiszár 对偶性)**. 广义的 $\phi$-散度（即使 $\mathrm{P} \nless \hat{\mathrm{P}}$，也可以取有限值）在 P 和 $\hat{\mathrm{P}}$ 交换时是不变的。形式上，我们有 $\mathrm{D}_\phi(\mathbb{P}, \hat{P}) = \mathrm{D}_\phi(\hat{\mathbb{P}}, \mathbb{P})$，其中 $\psi$ 表示 $\phi$ 的 Csiszár 对偶，通过 $\psi(s) = \phi^\pi(1, s) = t \phi(1 / t)$ 定义（Ben-Tal, Ben-Israel 和 Teboulle 1991, 引理 2.3）。可以验证，如果 $\phi$ 是定义 2.4 中所定义的有效熵函数，那么 $\psi$ 也是一个有效熵函数。这一关系表明，尽管 $\phi$-散度通常是不对称的，但通过聚焦于形式为 (2.10) 的散度模糊集，其中名义分布 P 作为散度的第二个参数，我们并没有牺牲一般性。从 Proposition 2.6 之后的讨论中可以看出，如果 $\phi^{\infty}(1) = \infty$，那么所有在 $\phi$-散度模糊集 (2.10) 中的分布 $P$ 满足 $\mathbb{P} \ll \hat{\mathbb{P}}$。类似地，如果 $\phi$ 的 Csiszár 对偶满足 $\psi^{\infty}(1) = \infty$，那么所有在 $\phi$-散度模糊集中的分布 P 满足 $\hat{P} \ll \mathbb{P}$。

表 2.1 列出了常见的熵函数及其 Csiszár 对偶。我们强调，Cressie-Read 散度家族包括 Pearson $\chi^2$-散度（当 $\beta = 2$）、Kullback-Leibler 散度（当 $\beta \rightarrow 1$）以及似然散度（当 $\beta \rightarrow 0$）作为特例。

DRO 文献通常关注于受限的 $\phi$-散度模糊集

$$
\mathcal{P} = \left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{P} \ll \hat{\mathrm{P}}, \mathrm{D}_\phi(\mathbb{P}, \hat{\mathbb{P}}) \leq r\right\}
$$

该模糊集由 Ben-Tal 等人（2013）引入。与标准的 $\phi$-散度模糊集（2.10）不同，它仅包含相对于参考分布 $\hat{\mathbb{P}}$ 完全连续的分布，即使 $\phi^{\infty}(1) < \infty$。Ben-Tal 等人（2013）研究了在受限的 $\phi$-散度模糊集上的 DRO 问题，假设参考分布 $\hat{\mathbb{P}}$ 是离散的。在这种情况下，绝对连续性约束 $\mathbb{P} \ll \hat{\mathbb{P}}$ 确保模糊集仅包含支持在 $\hat{\mathbb{P}}$ 原子上的离散分布，因此自然的最坏情况期望问题转化为一个有限的凸规划问题。Ben-Tal 等人（2013）进一步发展了这一问题类的对偶性理论。Shapiro（2017）将这一对偶性理论扩展到一般的参考分布 $\hat{\mathbb{P}}$，这些参考分布不一定是离散的。Hu, Hong 和 So（2013）以及 Jiang 和 Guan（2016）表明，任何受限的 $\phi$-散度模糊集上的分布鲁棒单个机会约束都等价于参考分布 $\hat{\mathbb{P}}$ 下的经典机会约束，只是置信度水平进行了缩放。Bayraksan 和 Love（2015）提供了对各种 $\phi$-散度的分类，并分析了对应的 $\phi$-散度模糊集的结构性质，假设 $\mathcal{Z}$ 是有限的。下面我们回顾标准和受限的 $\phi$-散度模糊集的常见实例。



| Divergence                        | $\phi(s)~(s \geq 0)$                                  | $\psi(s)~(s \geq 0)$                                  | $\phi^\infty(1)$ | $\psi^\infty(1)$   |
|------------------------------------|-------------------------------------------------------|-------------------------------------------------------|------------------|---------------------|
| **Kullback-Leibler**               | $s \log(s) - s + 1$                                   | $- \log(s) + s - 1$                                   | $\infty$         | 1                   |
| **Likelihood**                     | $-\log(s) + s - 1$                                   | $s \log(s) - s + 1$                                   | 1                | $\infty$       |
| **Total variation**                | $\frac{1}{2}\|s - 1\|$  | $\frac{1}{2}\|s - 1\|$ | $\frac{1}{2}          $     | $\frac{1}{2}          $   |
| **Pearson $\chi^2$**               | $(s - 1)^2$                                           | $\frac{1}{s} (s - 1)^2$                               | $\infty$         | 1                   |
| **Neyman $\chi^2$**                | $\frac{1}{s} (s - 1)^2$                               | $(s - 1)^2$                                           | 1                | $\infty$           |
| **Cressie-Read for $\beta \in (0, 1)$** | $\frac{s^\beta - \beta s + \beta - 1}{\beta (\beta - 1)}$ | $s^\beta - \beta s + \beta - 1$                       | $\frac{1}{1 - \beta}$ | $\frac{1}{\beta}$ |
| **Cressie-Read for $\beta > 1$**   | $\frac{s^\beta - \beta s + \beta - 1}{\beta (\beta - 1)}$ | $s^\beta - \beta s + \beta - 1$                       | $\infty$         | $\frac{1}{\beta - 1}$ |


表 2.1. 熵函数及其 Csiszár 对偶的示例

### 2.2.1. Kullback-Leibler 模糊集

Kullback-Leibler 散度是对应于熵函数的 $\phi$-散度，该熵函数满足 $\phi(s) = s \log(s) - s + 1$，适用于所有 $s \geq 0$；详见表 2.1。由于 $\phi^{\infty}(1) = +\infty$，因此它具有以下等价定义。

**定义 2.8 (Kullback-Leibler 散度)**. 对于 $\mathbb{P} \in \mathcal{P}(\mathcal{Z})$ 和 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$，Kullback-Leibler 散度定义为

$$
\mathrm{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \begin{cases} 
\int_{\mathcal{Z}} \log \left(\frac{\mathrm{d} \mathbb{P}}{d \hat{\mathbb{P}}}(z)\right) \mathrm{d} \mathbb{P}(z) & \text{如果 } \mathbb{P} \ll \hat{\mathbb{P}}, \\
+\infty & \text{否则.}
\end{cases}
$$

我们现在回顾一下 Kullback-Leibler 散度的著名变分公式。

**命题 2.9 (Donsker 和 Varadhan (1983))**. 对于 $\mathbb{P}$ 和 $\hat{\mathbb{P}}$，Kullback-Leibler 散度满足

$$
\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup_{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{d} \mathbb{P}(z) - \log \left(\int_{\mathcal{Z}} e^{f(z)} \mathrm{d} \hat{\mathbb{P}}(z)\right),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \mathbb{R}^d$ 的集合。

**证明**. 诱导 Kullback-Leibler 散度的熵函数 $\phi$ 的凸对偶满足 $\phi^*(t) = \exp(t) - 1$，其定义域为 $\mathbb{R}$。因此，命题 2.6 中建立的泛 $\phi$-散度的对偶表示式意味着

$$
\mathrm{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup_{f \in \mathcal{F}} \int_{\mathcal{Z}} f(z) \mathrm{dP}(z) - \int_{\mathcal{Z}}\left(\mathrm{e}^{f(z)} - 1\right) \mathrm{d} \hat{\mathbb{P}}(z),
$$

其中 $\mathcal{F}$ 表示所有有界的 Borel 函数 $f: \mathcal{Z} \rightarrow \mathbb{R}$ 的集合。注意，$\mathcal{F}$ 在常数平移下是不变的。也就是说，如果 $f(z)$ 是有界的 Borel 函数，那么对于任意常数 $c \in \mathbb{R}$，$f(z) + c$ 也是有界的 Borel 函数。因此，在上述最大化问题中，我们可以在 $f \in \mathcal{F}$ 和 $c \in \mathbb{R}$ 上同时优化，从而得到

$$
\operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) = \sup_{f \in \mathcal{F}} \sup_{c \in \mathbb{R}} \int_{\mathcal{Z}}(f(z)+c) \mathrm{d\mathbb{P}}(z) - \int_{\mathcal{Z}}\left(\mathrm{e}^{f(z)+c}-1\right) \mathrm{d} \hat{\mathbb{P}}(z).
$$

对于任何固定的 $f \in \mathcal{F}$，关于 $c$ 的内部最大化问题的最优解是

$$
c^{\star} = -\log \left(\int_{\mathcal{Z}} e^{f(z)} \mathrm{d} \hat{\mathbb{P}}(z)\right).
$$

将这个表达式代入目标函数得到（2.12）。命题 2.9 建立了 Kullback-Leibler 散度与熵风险度量之间的联系。这个联系将在第 4.3 节中变得有用。

Kullback-Leibler 模糊集的半径为 $r \geq 0$，围绕 $\hat{\mathbb{P}} \in \mathcal{P}(\mathcal{Z})$ 给出为

$$
\mathcal{P} = \{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \operatorname{KL}(\mathbb{P}, \hat{\mathbb{P}}) \leq r\}.
$$

由于 $\phi^{\infty}(1) = +\infty$，因此所有的分布 $\mathbb{P} \in \mathcal{P}$ 都是相对于 $\hat{\mathbb{P}}$ 完全连续的。因此，$\mathcal{P}$ 与受限的 Kullback-Leibler 模糊集是相同的。El Ghaoui 等人（2003）推导了当 $\hat{\mathbb{P}}$ 是高斯分布时，线性损失函数的最坏情况价值-at-risk 的封闭式表达式。Hu 和 Hong（2013）使用类似的方法表明，任何受限的 Kullback-Leibler 模糊集下的分布鲁棒个体机会约束都等价于参考分布 $\hat{\mathbb{P}}$ 下的经典机会约束，只是置信度水平被缩放了。Calafiore（2007）研究了当 $\hat{\mathbb{P}}$ 是离散分布时，最坏情况的均值-风险投资组合选择问题。Kullback-Leibler 模糊集还在最小二乘估计（Levy 和 Nikoukhah 2004）、假设检验（Levy 2008，Gül 和 Zoubir 2017）、滤波（Levy 和 Nikoukhah 2012，Zorzi 2016，2017a,b）、风险度量理论（Ahmadi-Javid 2012，Postek 等 2016）和极值分析（Blanchet, He 和 Murthy 2020）等方面找到了应用。
